## Part 1

In [34]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [35]:

# data from internet
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text

# use beautiful soup to phrase the HTML/XML 
soup = BeautifulSoup(raw_wikipedia_page,'xml')


In [37]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
toronto_df = pd.DataFrame(data = data,columns = columns)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [38]:
#list to Pandas DataFrame
toronto_df = pd.DataFrame(data = data,columns = columns)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [39]:
#Remove Boroughs that are 'Not assigned'
toronto_df = toronto_df[toronto_df["Borough"] != 'Not assigned']
toronto_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [40]:
# More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
toronto_df["Neighborhood"] = toronto_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
toronto_df = toronto_df.drop_duplicates()

#update index to be postcode if it isn't already
if(toronto_df.index.name != "Postal Code"):
    toronto_df = toronto_df.set_index("Postal Code")
    
toronto_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [54]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
toronto_df["Neighborhood"].replace("Not assigned", toronto_df["Borough"],inplace=True)
toronto_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [42]:

toronto_df.shape

(103, 2)

## Part 2

In [56]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
coors = pd.read_csv('toronto_coordinates.csv')
coors = coors.set_index("Postal Code")
coors.head()


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [58]:
toronto_coors_df = pd.concat([toronto_df, coors], axis=1, join='inner')

toronto_coors_df.index.name = "Postal Code"
toronto_coors_df.reset_index(inplace=True)

print(toronto_coors_df.shape)
toronto_coors_df.head()

(103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
